In [1]:
import pandas as pd
from src import Utils, LSTM
from skorch.dataset import ValidSplit
from torch.nn import BCEWithLogitsLoss

/tmp/ipykernel_29295/4062262799.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_md' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)
/home/nate/minicon

In [2]:
MODEL_FOLDER = 'model_lstm'

In [3]:
dataset = Utils.read_csv_file('datasets/datasetall.csv')
dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [4]:
Utils.seed_random_number_generators()

Random number generators seeded.


In [5]:
TEST_SIZE = 0.1

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

In [6]:
X_train

0        pag hindi nanalo si Norberto Gonzales pwede ba...
1        Ngayon lang ako super proud sa PRESIDENTE na i...
2        JUST SAW SOMEONE CALL BBM BLENGBLONG HAHAHAHAH...
3        Rep. Binay on her leadership style: I am very ...
4        Liwanag o dilim? May oras pa. Kakampink Leni L...
                               ...                        
25611    "Kala ko wala andito pala si Marcos."*pertaini...
25612    cathy [USERNAME] Dec [USERNAME] parang tanga i...
25613                             Nognog+pandak= BINAY ftw
25614    BINAY:Did your enormous wealth all come from y...
25615                                Uunlad tayo kay Binay
Name: text, Length: 25616, dtype: object

In [7]:
y_train

0        1
1        0
2        1
3        0
4        0
        ..
25611    0
25612    1
25613    1
25614    1
25615    0
Name: label, Length: 25616, dtype: int64

In [8]:
X_test

0       PRESIDENTE DUTERTE I'm sure in last debateitao...
1       CHANGE IS BADLY NEEDED No To Mar Roxas2016 Dut...
2                                One Pink March Leni Kiko
3                               see youuu later Leni Kiko
4       [USERNAME] Nangyari na yan eh pero kahit anong...
                              ...                        
2840    kaya siguro umabot ng milyon yung boto kay MAR...
2841    Dedicating my 21km run for my chosen Presand V...
2842    Bakit si Mar? Because DuterteGrace Poe and VP ...
2843    patalo po ung patalastas ni Mar Roxas....malas...
2844    Kapihan with Sen. Bongbong Marcos startshe say...
Name: text, Length: 2845, dtype: object

In [9]:
y_test

0       0
1       0
2       0
3       0
4       0
       ..
2840    1
2841    0
2842    1
2843    1
2844    0
Name: label, Length: 2845, dtype: int64

In [10]:
epochs = [100, 200, 300]
learning_rate = [0.1, 0.01, 0.001, 0.0001, 0.00001]
batch_size = [32, 64, 128]
hidden_size = [100, 200, 300, 400]
num_layers = [1, 2, 3]

train_lstm = LSTM.LstmPipeline

In [11]:
train_lstm.set_params(
  lstm__train_split=None,
  # lstm__module__input_size=300,
  lstm__module__hidden_size=250,
  lstm__module__num_layers=1,
  lstm__module__output_size=2,
  lstm__optimizer__lr=0.015,
  lstm__max_epochs=30,
  lstm__batch_size=32,
  tokenizer__remove_stopwords=False,
  # lstm__criterion=BCEWithLogitsLoss,
)

train_lstm['lstm'].callbacks[0].dirname = f'{MODEL_FOLDER}/train_lstm'
train_lstm['lstm'].callbacks[0].monitor = 'train_loss_best'

In [12]:
train_lstm

Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'src.LSTM.LstmModel'>,
  module__hidden_size=250,
  module__input_size=300,
  module__num_layers=1,
  module__output_size=2,
))])

In [13]:
train_lstm.fit(X_train, y_train)

  0%|          | 0/801 [00:00<?, ?it/s]

  epoch    train_loss    cp     dur
-------  ------------  ----  ------
      1        0.4881     +  5.7128


  0%|          | 0/801 [00:00<?, ?it/s]

      2        0.4351     +  5.5462


  0%|          | 0/801 [00:00<?, ?it/s]

...

  0%|          | 0/801 [00:00<?, ?it/s]

     69        0.3505     +  5.4143


  0%|          | 0/801 [00:00<?, ?it/s]

     70        0.3501     +  6.0770


Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(300, 250, batch_first=True)
    (fc): Linear(in_features=250, out_features=2, bias=True)
  ),
))])

In [14]:
accuracy, recall, precision, f1 = Utils.get_prediction_results(
  X_test,
  y_test,
  train_lstm,
)

Accuracy: 0.8028119507908612
Recall: 0.8186889818688982
Precision: 0.7959322033898305
F1-score: 0.8071502234444826


In [15]:
Utils.save_trained_model(train_lstm, f"{MODEL_FOLDER}/LSTM")

Ensemble model saved to Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(300, 250, batch_first=True)
    (fc): Linear(in_features=250, out_features=2, bias=True)
  ),
))]).pkl


In [16]:
history_data_frame = pd.DataFrame(
    train_lstm['lstm'].history
).set_index('epoch')
history_data_frame.to_csv(f'{MODEL_FOLDER}/lstm_history.csv')

In [17]:
history_data_frame

,batches,train_batch_count,dur,train_loss,train_loss_best,event_cp
epoch,,,,,,
1,"[{'train_loss': 0.7008848786354065, 'train_bat...",801,5.712763,0.488105,True,True
2,"[{'train_loss': 0.5208430290222168, 'train_bat...",801,5.546200,0.435128,True,True
3,"[{'train_loss': 0.549223780632019, 'train_batc...",801,5.442185,0.422317,True,True
4,"[{'train_loss': 0.5465188026428223, 'train_bat...",801,5.528706,0.415251,True,True
5,"[{'train_loss': 0.542667806148529, 'train_batc...",801,5.414896,0.410666,True,True
...,...,...,...,...,...,...
66,"[{'train_loss': 0.3867233693599701, 'train_bat...",801,5.485579,0.351296,True,True
67,"[{'train_loss': 0.4018794298171997, 'train_bat...",801,5.484462,0.352043,False,False
68,"[{'train_loss': 0.4243532419204712, 'train_bat...",801,5.421669,0.351350,False,False


In [18]:
metrics_data_frame = pd.DataFrame([{
  'accuracy': accuracy,
  'recall': recall,
  'precision': precision,
  'f1': f1,
}])
metrics_data_frame.to_csv(f'{MODEL_FOLDER}/lstm_metrics.csv')